## Pipeline Tutorial

### install

`Pipeline` is distrubuted alone with [fate_client](https://pypi.org/project/fate-client/).

```bash
pip install fate_client
```

Since `Pipeline` is only a clien of `FATE`, we need some configuration to indicate which `FATE Flow Service` to connect to. Once `fate_client` installed, one can find an cmd enterpoint name `pipeline`:

In [1]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  init   - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [2]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


### homo nn

The `pipeline` package provides many useful components to compose an `FATE pipeline`.

In [3]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component.dataio import DataIO
from pipeline.component.reader import Reader
from pipeline.component.homo_nn import HomoNN
from pipeline.interface.data import Data

Make a `pipeline` instance:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: [10000, 9999]
        * arbiter: 9999
    

In [4]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=9999) \
        .set_roles(guest=9999, host=[10000], arbiter=10000)

Define a `Reader` to load data

In [5]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=9999).component_param(
    table={"name": "breast_homo_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=10000).component_param(
    table={"name": "breast_homo_host", "namespace": "experiment"})

Add an `DataIo` to parse data into data instance

In [6]:
dataio_0 = DataIO(name="dataio_0", with_label=True)
# set guest parameter
dataio_0.get_party_instance(role='guest', party_id=9999).component_param(
    with_label=True)
dataio_0.get_party_instance(role='host', party_id=[10000]).component_param(
    with_label=True)

Now, we define the `HomoNN` component.

In [7]:
homo_nn_0 = HomoNN(
    name="homo_nn_0", 
    max_iter=10, 
    batch_size=-1, 
    early_stop={"early_stop": "diff", "eps": 0.0001})

Add single `Dense` layer:

In [8]:
from tensorflow.keras.layers import Dense
homo_nn_0.add(
    Dense(units=1, input_shape=(10,), activation="sigmoid"))

Compile:

In [9]:
from tensorflow.keras import optimizers
homo_nn_0.compile(
    optimizer=optimizers.Adam(learning_rate=0.05), 
    metrics=["accuracy", "AUC"],
    loss="binary_crossentropy")

Add components to pipeline:

    - dataio_0 comsume reader_0's output data
    - homo_nn_0 comsume dataio_0's output data

In [10]:
pipeline.add_component(reader_0)
pipeline.add_component(dataio_0, data=Data(data=reader_0.output.data))
pipeline.add_component(homo_nn_0, data=Data(train_data=dataio_0.output.data))
pipeline.compile();

We are almost ready to submit our pipeline job except for `JobParameters`

In [11]:
from pipeline.runtime.entity import JobParameters
job_parameters = JobParameters(backend=0, work_mode=0)

Now, submit(fit) out pipeline:

In [12]:
pipeline.fit(job_parameters)

2020-11-02 17:39:31.756 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:121 - Job id is 2020110217393142628946


Job is still waiting, time elapse: 0:00:00
Running component reader_0, time elapse: 0:00:02
Running component dataio_0, time elapse: 0:00:04


2020-11-02 17:39:50.461 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:129 - Job is success!!! Job id is 2020110217393142628946
2020-11-02 17:39:50.462 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:130 - Total time: 0:00:18


Success! Now we can get summary from homo_nn_0:

In [13]:
pipeline.get_component("homo_nn_0").get_summary()

{'is_converged': False,
 'loss_history': [0.9953389167785645,
  0.6466653943061829,
  0.4520302712917328,
  0.34566518664360046,
  0.2823573052883148,
  0.23998518288135529,
  0.20845219492912292,
  0.18313930928707123,
  0.16189616918563843,
  0.1436993032693863]}

For more examples about using pipeline to submit `HomoNN` jobs, please refer to [HomoNN Examples](https://github.com/FederatedAI/FATE/tree/master/examples/pipeline/homo_nn)